In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
import json

load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("AZURE_OPENAI_VERSION")
)
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")

In [2]:
prompt = """Your task is to create product descriptions for a retail e-commerce shop.
Pick a category of product from the list below and create a product description for it.
Be very create for the product description and make sure it is unique and engaging.
The product should have a name, a category, a brand, a price, a description, and a list of 5 tags that describe the product.
- "Trouser"
- "Computer equipment"
- "Televisions"
- "Shovels"
- "Toy cars"
- "Lego sets"
- "Sports Equipment"
- "Shoes"
- "Jewelry"
- "Tablets"
- "Handbags"
- "Watches"
- "Tableware"
- "Backpacks"
- "Sofas"
- "Musical Instruments"
- "Chairs"
- "Pull-over"
- "Jackets"
- "T-shirts"
- "Sweatshirts"
- "Dresses"
- "Washing machines"
- "Refrigerators"
- "Microwaves"
- "Bicycles"
- "Speakers"

The output should be a single json object like this:
{
  "id": 1,
  "category": "Clothing",
  "name": "Stylish Denim Jacket",
  "brand": "Fashionista",
  "price": 49.99,
  "description": "This stylish denim jacket is the perfect addition to your wardrobe. Made from high-quality denim, it features a classic design with a modern twist. The jacket is perfect for layering over your favorite tops and dresses, and the relaxed fit makes it comfortable to wear all day long. Whether you're running errands or meeting friends for brunch, this jacket will keep you looking chic and stylish.",
  "tags": ["denim", "jacket", "fashion", "stylish", "casual"]
}

Here are a few examples of product descriptions to get you started:
{
  "id": 1,
  "category": "Outdoor Gear",
  "name": "Camping Tent",
  "brand": "Adventure Gear",
  "price": 199.99,
  "description": "This camping tent is perfect for your next outdoor adventure. It's easy to set up and take down, and it's made from durable materials that will keep you dry and comfortable in any weather. The tent has plenty of room for you and your gear, and the mesh windows and doors provide plenty of ventilation. Whether you're camping in the mountains or by the beach, this tent is the perfect choice for your next trip.",
  "tags": ["camping", "tent", "outdoor", "adventure", "gear"]  
}

"""

svgPrompt = """Your task is to create a SVG image of a product for a retail e-commerce shop. 
Keep the image as simple as possible, using basic shapes and colors to represent the product. Review the SVG documentation for more information on how to create SVG images.

Here is an example of an SVG image for a product:
<svg width="200" height="200" viewBox="0 0 200 150" xmlns="http://www.w3.org/2000/svg"><rect x="20" y="20" width="160" height="100" fill="#000000"/><rect x="30" y="30" width="140" height="80" fill="#1e90ff"/><rect x="90" y="120" width="20" height="10" fill="#000000"/></svg>  

The output should be only the SVG code for the image without comments or line breaks.

The picture should have be a 100x100 square and should represent this product:
"""

products = []

target = 400
index = 200
while(index < target):
    print("Number of products: ", len(products))
    index = index + 1
    print(index)

    try:
        response = client.chat.completions.create(
          model = model_name, 
          temperature=1.0,
          messages = [{"role" : "assistant", "content" : prompt}],
        )
        productstring = response.choices[0].message.content
        print(productstring)
        print("parsing json")
        productstring = productstring.replace("json", "")
        productstring = productstring.replace("```", "")
        productstring = productstring.replace("\n", "")
        print(productstring)
        json_product = json.loads(productstring)
        json_product["id"] = index

        svgPrompt = svgPrompt + " " + json_product["name"]
        print("Creating SVG for product: ", json_product["name"])
        
        svgResponse = client.chat.completions.create(
                        model = model_name, 
                        temperature=1.0,
                        messages = [{"role" : "assistant", "content" : svgPrompt}],
                    )
        print(svgResponse)
        svgResponseJon = svgResponse.choices[0].message.content
        svgResponseJon = svgResponseJon.replace("json", "")
        svgResponseJon = svgResponseJon.replace("```", "")
        svgResponseJon = svgResponseJon.replace("svg<svg", "<svg")        
        svgResponseJon = svgResponseJon.replace("\n", "")
        svgResponseJon = svgResponseJon.replace("\"", "'")
        print(svgResponseJon)
        json_product["svg"] = svgResponseJon
        print(json_product)
        products.append(json_product)
    except Exception as e:
        print("Failed to parse JSON", e)
        print(productstring)


with open('products.json', 'w') as f:
    json.dump(products, f)

Number of products:  0
201
{
  "id": 36,
  "category": "Computer equipment",
  "name": "SonicWave Ultra 4K Monitor",
  "brand": "TechVision",
  "price": 329.99,
  "description": "Experience stunning visuals like never before with the SonicWave Ultra 4K Monitor by TechVision. Immerse yourself in breathtaking clarity and vibrant colors, thanks to its 3840 x 2160 resolution and IPS display technology. Ideal for gaming, professional design, or binge-watching your favorite shows, this monitor transforms your work and play space with its slim bezel design and versatile connectivity options, including HDMI, DisplayPort, and USB-C. Eye Comfort technology reduces strain, making it perfect for long hours of usage. Elevate your digital experience with the SonicWave Ultra 4K Monitor today.",
  "tags": ["monitor", "4K", "technology", "gaming", "display"]
}
parsing json
{  "id": 36,  "category": "Computer equipment",  "name": "SonicWave Ultra 4K Monitor",  "brand": "TechVision",  "price": 329.99,  "